In [0]:
#Imports
from bs4 import BeautifulSoup as bs 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [0]:
msa_counties = pd.read_csv('MSA Counties.csv')
county_education = pd.read_csv('County Education.csv')

In [0]:
clean_msa = msa_counties.drop(msa_counties.columns[1], axis =1)
clean_msa.rename({clean_msa.columns.values[0]:'MSA_Code', clean_msa.columns.values[3]: 'County2'}, axis =1, inplace = True)

In [0]:
new_df_county2 = []
len_msa = len(clean_msa)
for i in range(len_msa):
    if pd.isnull(clean_msa.loc[i, 'MSA']):
        clean_msa.loc[i,'MSA'] = clean_msa.loc[i-1,'MSA']
        clean_msa.loc[i, 'MSA_Code'] = clean_msa.loc[i-1, 'MSA_Code']
    if pd.notnull(clean_msa.loc[i,'County2']):
        new_county = clean_msa.loc[i,'County2']
        c_MSA_code = clean_msa.loc[i, 'MSA_Code']
        c_MSA = clean_msa.loc[i, 'MSA']
        new_df_county2.append([c_MSA_code, c_MSA, new_county])
temp_df = pd.DataFrame(new_df_county2).rename({0: 'MSA_Code', 1: 'MSA', 2: 'County'}, axis =1)

In [0]:
clean_msa = clean_msa.append(temp_df).drop('County2', axis=1)
regex = r",[\s\S]*$"
state = []
for x in clean_msa['MSA']:
    state.append(re.findall(regex, x)[0].strip(',').strip(' '))

clean_msa['State'] = state

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
clean_msa.reset_index(inplace = True)

In [0]:
clean_msa.head()

,index,County,MSA,MSA_Code,State
0,0,Taylor County,"Abilene, TX",40.0,TX
1,1,Portage County,"Akron, OH",80.0,OH
2,2,Summit County,"Akron, OH",80.0,OH
3,3,Dougherty County,"Albany, GA",120.0,GA
4,4,Lee County,"Albany, GA",120.0,GA


In [0]:
#drop state names in county
states_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
states_list = [x + ':' for x in states_list]
state_index = []
for i in range(len(clean_msa)):
    if clean_msa.loc[i, 'County'] in states_list:
        state_index.append(i)
clean_msa.drop(state_index, inplace=True)

In [0]:
clean_msa = clean_msa[['MSA', 'MSA_Code','State', 'County']].sort_values('MSA').reset_index()

In [0]:
#function to delete the remove words in counties
def del_county(county_list):
    clean_county = []
    remove = ['County', 'Borough', 'Parish']
    for x in county_list:
        if any(word in x for word in remove):
            clean_county.append(x.rsplit(' ', 1)[0])
        else:
            clean_county.append(x)
    return clean_county
#convert str to float
def str_to_float(series):
    floats = []
    for x in series:
        if type(x) == str:
            f = float(x.replace(',', ''))
            floats.append(f)
        else:
            floats.append(x)
    return floats

In [0]:
#Delete state names in county
clean_county = []
no_state = []
regex_s = r',[\s\S]*$'
for x in clean_msa['County']:
    no_state.append(re.sub(regex_s,"", str(x)))
clean_msa['County'] = no_state

#Use del_county on clean_msa
no_county = del_county(clean_msa['County'])
clean_msa['County'] = no_county


In [0]:
clean_education = county_education.loc[:,["State", "Area name", "Bachelor's degree or higher, 2000", "Bachelor's degree or higher, 2013-17"]]
clean_education.rename({"Area name": 'County'},axis =1, inplace = True)
clean_education.dropna(inplace=True)
clean_education["Bachelor's degree or higher, 2000"] = str_to_float(clean_education["Bachelor's degree or higher, 2000"])
clean_education["Bachelor's degree or higher, 2013-17"] = str_to_float(clean_education["Bachelor's degree or higher, 2013-17"])

In [0]:
#set counties to only name w/o county, borough, parish
only_name = del_county(clean_msa['County'])
clean_msa['County'] = only_name
clean_msa

,index,MSA,MSA_Code,State,County
0,0,"Abilene, TX",40.0,TX,Taylor
1,1,"Akron, OH",80.0,OH,Portage
2,2,"Akron, OH",80.0,OH,Summit
3,3,"Albany, GA",120.0,GA,Dougherty
4,4,"Albany, GA",120.0,GA,Lee
5,702,"Albany/Schenectady/Troy, NY",160.0,NY,Schoharie
6,701,"Albany/Schenectady/Troy, NY",160.0,NY,Schenectady
7,700,"Albany/Schenectady/Troy, NY",160.0,NY,Saratoga
8,6,"Albany/Schenectady/Troy, NY",160.0,NY,Montgomery
9,7,"Albany/Schenectady/Troy, NY",160.0,NY,Rensselaer


In [0]:
#Business Patterns (2000) by county

business_2000_county = pd.read_csv("cbp00co.txt")

business_2000_county2 = business_2000_county.loc[:, ["fipstate", "fipscty", "est"]].rename(columns={"fipstate":
                                                                                                    "State Code", 
                                                                                                    "fipscty":"County Code", 
                                                                                                   "est":"Total Number of Enterprises"})

business_2000_county3 = business_2000_county2.groupby(["State Code", "County Code"]).sum()

business_2000_county4 = business_2000_county3.reset_index()

business_2000_county4.head()

,State Code,County Code,Total Number of Enterprises
0,1,1,4574
1,1,3,23040
2,1,5,3418
3,1,7,1970
4,1,9,4432


In [0]:
#Business Patterns (2010) by county

business_2010_county = pd.read_csv("cbp10co.txt")

business_2010_county2 = business_2010_county.loc[:, ["fipstate", "fipscty", "est"]].rename(columns={"fipstate":
                                                                                                    "State Code", 
                                                                                                    "fipscty":"County Code", 
                                                                                                   "est":"Total Number of Enterprises"})

business_2010_county3 = business_2010_county2.groupby(["State Code", "County Code"]).sum()

business_2010_county4 = business_2010_county3.reset_index()

business_2010_county4.head()

,State Code,County Code,Total Number of Enterprises
0,1,1,5222
1,1,3,27954
2,1,5,3088
3,1,7,1754
4,1,9,4198


In [0]:
#Merging together businesses features by county

businesses_feature_df =  business_2000_county4.merge(business_2010_county4, on = ["State Code", "County Code"], how = "left")

businesses_feature_df2= businesses_feature_df.rename(columns={"Total Number of Enterprises_x":"Total Number of Enterprises (2000)", 
                                                             "Total Number of Enterprises_y":"Total Number of Enterprises (2010)"})

businesses_feature_df2["Quantitative Change"] = businesses_feature_df2["Total Number of Enterprises (2010)"] - businesses_feature_df2["Total Number of Enterprises (2000)"]

businesses_feature_df3 = businesses_feature_df2.drop(columns=["Quantitative Change"])
                                        

businesses_feature_df4 = businesses_feature_df3

businesses_feature_df4.head()

,State Code,County Code,Total Number of Enterprises (2000),Total Number of Enterprises (2010)
0,1,1,4574,5222.0
1,1,3,23040,27954.0
2,1,5,3418,3088.0
3,1,7,1970,1754.0
4,1,9,4432,4198.0


In [0]:
#Working with FIPS Codes dataset
fips_codes_df = pd.read_csv("US_FIPS_Codes.csv")

fips_codes_df2 = fips_codes_df.drop(fips_codes_df.index[0])

fips_codes_df3 = fips_codes_df2.rename(columns={"U.S. FIPS County Codes":"State Name", "Unnamed: 1": "County Name", 
                                               "Unnamed: 2":"State Code", "Unnamed: 3": "County Code"})

fips_codes_df3["State Name"] = fips_codes_df3["State Name"].map({"Alabama":"AL", "Alaska":"AK", "Arizona":"AZ", 
                                                                 "Arkansas":"AR", "California":"CA", "Colorado":"CO", 
                                                                "Connecticut":"CT", "Delaware":"DE", "Florida":"FL", 
                                                                 "Georgia":"GA", "Hawaii":"HI", "Idaho":"ID", "Illinois":"IL", 
                                                                "Indiana":"IN", "Iowa":"IA", "Kansas":"KS", "Kentucky":"KY", 
                                                                "Louisiana":"LA", "Maine":"ME", "Maryland":"MD", "Massachusetts":"MA", 
                                                                "Michigan":"MI", "Minnesota":"MN", "Mississippi":"MS", "Missouri":"MO",
                                                                "Montana":"MT", "Nebraska":"NE", "Nevada":"NV", "New Hampshire":"NH", 
                                                                "New Jersey":"NJ", "New Mexico":"NM", "New York":"NY", "North Carolina":"NC", 
                                                                "North Dakota":"ND", "Ohio":"OH", "Oklahoma":"OK", "Oregon":"OR", "Pennsylvania":"PA",
                                                                "Rhode Island":"RI", "South Carolina":"SC", "South Dakota":"SD", "Tennessee":"TN", 
                                                                "Texas":"TX", "Utah":"UT", "Vermont":"VT", "Virginia":"VA", "Washington":"WA", "West Virginia":"WV", 
                                                                "Wisconsin":"WI", "Wyoming":"WY"})

fips_codes_df4 = fips_codes_df3

fips_codes_df4.head()

,State Name,County Name,State Code,County Code
1,AL,Autauga,01,001
2,AL,Baldwin,01,003
3,AL,Barbour,01,005
4,AL,Bibb,01,007
5,AL,Blount,01,009


In [0]:
fips_codes_df4["State Code"] = fips_codes_df4["State Code"].astype(int)
fips_codes_df4["County Code"] = fips_codes_df4["County Code"].astype(int)

fips_codes_df5 = fips_codes_df4

fips_codes_df5.head()

,State Name,County Name,State Code,County Code
1,AL,Autauga,1,1
2,AL,Baldwin,1,3
3,AL,Barbour,1,5
4,AL,Bibb,1,7
5,AL,Blount,1,9


In [0]:
final_businesses_df1 = businesses_feature_df4.merge(fips_codes_df5, on = ["State Code", "County Code"], how = "left")

final_businesses_df2 = final_businesses_df1

final_businesses_df3 = final_businesses_df2.drop(columns=["State Code", "County Code"])

final_businesses_df4 = final_businesses_df3.rename(columns={"State Name":"State", "County Name":"County"})

final_businesses_df4.head()

,Total Number of Enterprises (2000),Total Number of Enterprises (2010),State,County
0,4574,5222.0,AL,Autauga
1,23040,27954.0,AL,Baldwin
2,3418,3088.0,AL,Barbour
3,1970,1754.0,AL,Bibb
4,4432,4198.0,AL,Blount


In [0]:
clean_education_county = del_county(clean_education['County'])

clean_education2 = clean_education.copy()
clean_education2['County'] = clean_education_county
educ_business = clean_education2.merge(final_businesses_df4, how = 'inner', on = ['County','State'])
educ_business

,State,County,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010)
0,AL,Autauga,4972.0,9176.0,4574,5222.0
1,AL,Baldwin,22146.0,43968.0,23040,27954.0
2,AL,Barbour,2068.0,2218.0,3418,3088.0
3,AL,Bibb,962.0,2097.0,1970,1754.0
4,AL,Blount,3235.0,5172.0,4432,4198.0
5,AL,Bullock,586.0,990.0,816,720.0
6,AL,Butler,1433.0,2253.0,2992,2526.0
7,AL,Calhoun,11265.0,14113.0,15288,14550.0
8,AL,Chambers,2339.0,3158.0,3834,3422.0
9,AL,Cherokee,1636.0,2344.0,2128,2136.0


In [0]:
clean_msa2 = clean_msa.merge(educ_business, how = 'left', on = 'County')
clean_msa2

,index,MSA,MSA_Code,State_x,County,State_y,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010)
0,0,"Abilene, TX",40.0,TX,Taylor,FL,1147.0,1390.0,2274.0,2446.0
1,0,"Abilene, TX",40.0,TX,Taylor,GA,476.0,652.0,838.0,752.0
2,0,"Abilene, TX",40.0,TX,Taylor,IA,571.0,623.0,992.0,834.0
3,0,"Abilene, TX",40.0,TX,Taylor,KY,1857.0,3111.0,4036.0,3678.0
4,0,"Abilene, TX",40.0,TX,Taylor,TX,16951.0,19767.0,20822.0,20170.0
5,0,"Abilene, TX",40.0,TX,Taylor,WV,1255.0,2281.0,1272.0,1322.0
6,0,"Abilene, TX",40.0,TX,Taylor,WI,1418.0,2030.0,2820.0,2776.0
7,1,"Akron, OH",80.0,OH,Portage,OH,19779.0,28529.0,18548.0,17782.0
8,1,"Akron, OH",80.0,OH,Portage,WI,9378.0,14080.0,9728.0,9678.0
9,2,"Akron, OH",80.0,OH,Summit,CO,7630.0,10694.0,10942.0,12568.0


In [0]:
clean_msa2['State_y'] = [str(x) for x in clean_msa2['State_y']]

In [0]:
#Get the right rows for each county
tf = []
for i in range(len(clean_msa2)):
    tf.append(clean_msa2.loc[i, 'State_y'] in clean_msa2.loc[i,'State_x'])
clean_msa2 = clean_msa2[tf]

In [0]:
clean_msa2

,index,MSA,MSA_Code,State_x,County,State_y,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010)
4,0,"Abilene, TX",40.0,TX,Taylor,TX,16951.0,19767.0,20822.0,20170.0
7,1,"Akron, OH",80.0,OH,Portage,OH,19779.0,28529.0,18548.0,17782.0
10,2,"Akron, OH",80.0,OH,Summit,OH,91096.0,119093.0,86042.0,81578.0
12,3,"Albany, GA",120.0,GA,Dougherty,GA,10347.0,12055.0,15376.0,14710.0
16,4,"Albany, GA",120.0,GA,Lee,GA,2550.0,4908.0,1216.0,2218.0
25,702,"Albany/Schenectady/Troy, NY",160.0,NY,Schoharie,NY,3586.0,4795.0,3410.0,3332.0
26,701,"Albany/Schenectady/Troy, NY",160.0,NY,Schenectady,NY,26218.0,32881.0,17616.0,18552.0
27,700,"Albany/Schenectady/Troy, NY",160.0,NY,Saratoga,NY,41678.0,64735.0,24602.0,29734.0
38,6,"Albany/Schenectady/Troy, NY",160.0,NY,Montgomery,NY,4606.0,5818.0,6738.0,6770.0
45,7,"Albany/Schenectady/Troy, NY",160.0,NY,Rensselaer,NY,23793.0,33046.0,16240.0,17854.0


In [0]:
#Population

pop_msa = pd.read_csv("population-change-metro-micro-areas.csv", encoding='iso-8859-1')
pop_msa.head()

,NAME,LSAD,CENSUSPOP2000,CENSUSPOP2010,NPOPCHG_2010,PPOPCHG_2010
0,NaN,NaN,NaN,NaN,NaN,NaN
1,Metropolitan Statistical Areas,NaN,NaN,NaN,NaN,NaN
2,"Abilene, TX",Metropolitan Statistical Area,160245.0,165252.0,5007.0,3.124590471
3,"Akron, OH",Metropolitan Statistical Area,694960.0,703200.0,8240.0,1.185679751
4,"Albany, GA",Metropolitan Statistical Area,157833.0,157308.0,-525.0,-0.332630058


In [0]:
#Cleaning the feature "population growth for MSAs"
pop_msa2 = pop_msa.drop(pop_msa.index[[0, 1]])

pop_msa3 = pop_msa2.iloc[0:366, :]

pop_msa4 = pop_msa3.drop(columns=["LSAD", "CENSUSPOP2000", "CENSUSPOP2010", "NPOPCHG_2010"])

pop_msa5 = pop_msa4.rename(columns={"NAME":"Metropolitan Statistical Area (MSA)", 
                                    "PPOPCHG_2010":"Population Change From 2000 to 2010 (%)"})

pop_msa6 = pop_msa5.rename(columns={"Metropolitan Statistical Area (MSA)": "MSA"})

pop_msa6.head()

,MSA,Population Change From 2000 to 2010 (%)
2,"Abilene, TX",3.124590471
3,"Akron, OH",1.185679751
4,"Albany, GA",-0.332630058
5,"Albany-Schenectady-Troy, NY",5.429514152
6,"Albuquerque, NM",21.5758536


In [0]:
asc = pd.read_csv("regiongdpchange.csv")
arr = []
for x in asc["Metropolitan Statistical Area"].values:
    arr.append(x.replace(" (Metropolitan Statistical Area)", ""))
asc["Metropolitan Statistical Area"] = arr
asc = asc.drop(['2002', '2003', '2004', '2005', '2006'], axis = 1)
asc['Percent Change'] = (asc["2007"].values - asc["2001"].values)/(asc["2001"])
asc = asc.drop(['2001', '2007', 'UYT'], axis = 1)
asc2 = asc.rename(columns={"Metropolitan Statistical Area":"MSA", "Percent Change":"Percent Change in GDP (2001 to 2007)"})
asc2.head()

,MSA,Percent Change in GDP (2001 to 2007)
0,"Abilene, TX",0.159867
1,"Akron, OH",0.087970
2,"Albany, GA",0.000753
3,"Albany, OR",0.009704
4,"Albany-Schenectady-Troy, NY",0.011663


In [0]:
pop_gdp_merged = pop_msa6.merge(asc2, on = "MSA", how = "left")
#replace - with / in MSa
msa_dash_slash = []
for x in pop_gdp_merged['MSA']:
    msa_dash_slash.append(x.replace('-','/'))
pop_gdp_merged['MSA'] = msa_dash_slash
pop_gdp_merged

,MSA,Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007)
0,"Abilene, TX",3.124590471,0.159867
1,"Akron, OH",1.185679751,0.087970
2,"Albany, GA",-0.332630058,0.000753
3,"Albany/Schenectady/Troy, NY",5.429514152,0.011663
4,"Albuquerque, NM",21.5758536,0.049326
5,"Alexandria, LA",6.127486469,0.118305
6,"Allentown/Bethlehem/Easton, PA/NJ",10.9101223,-0.013964
7,"Altoona, PA",-1.59124698,0.076935
8,"Amarillo, TX",10.31202267,0.104980
9,"Ames, IA",11.9540891,0.069906


In [0]:
only_msa =  clean_msa2.groupby('MSA').sum().reset_index()
clean_msa3 = only_msa.merge(pop_gdp_merged, on = "MSA", how = "left")
clean_msa3.head()

,MSA,index,MSA_Code,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010),Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007)
0,"Abilene, TX",0,40.0,16951.0,19767.0,20822.0,20170.0,3.124590471,0.159867
1,"Akron, OH",3,160.0,110875.0,147622.0,104590.0,99360.0,1.185679751,0.087970
2,"Albany, GA",7,240.0,12897.0,16963.0,16592.0,16928.0,-0.332630058,0.000753
3,"Albany/Schenectady/Troy, NY",2121,960.0,164967.0,224332.0,121318.0,132984.0,5.429514152,0.011663
4,"Albuquerque, NM",27,600.0,129503.0,189701.0,105264.0,110622.0,21.5758536,0.049326


In [0]:
established = ['Raleigh/Durham/Chapel Hill, NC', 'Boston, MA/NH', 'San Diego, CA','New York, NY',
      'Washington, DC/MD/VA/WV','Dallas, TX', 'Minneapolis/St. Paul, MN/WI', 'Denver, CO', 'Atlanta, GA', 'Seattle/Bellevue/Everett, WA',
      'Austin/San Marcos, TX', 'San Jose, CA', 'San Francisco, CA']
declined = ['Cleveland/Lorain/Elyria, OH','Flint, MI','Detroit, MI', 'Merced, CA','Akron, OH','Gary, IN','Scranton/Wilkes-Barre/Hazleton, PA',
           'Pittsburgh, PA', 'Toledo, OH', 'Cincinnati, OH/KY/IN', 'St. Louis, MO/IL','Visalia/Tulare/Porterville, CA','Syracuse, NY',
           'Youngstown/Warren, OH']


In [0]:
pop_gdp_merged.sort_values('MSA')['MSA'].tolist()

['Abilene, TX',
 'Akron, OH',
 'Albany, GA',
 'Albany/Schenectady/Troy, NY',
 'Albuquerque, NM',
 'Alexandria, LA',
 'Allentown/Bethlehem/Easton, PA/NJ',
 'Altoona, PA',
 'Amarillo, TX',
 'Ames, IA',
 'Anchorage, AK',
 'Anderson, IN',
 'Anderson, SC',
 'Ann Arbor, MI',
 'Anniston/Oxford, AL',
 'Appleton, WI',
 'Asheville, NC',
 'Athens/Clarke County, GA',
 'Atlanta/Sandy Springs/Marietta, GA',
 'Atlantic City/Hammonton, NJ',
 'Auburn/Opelika, AL',
 'Augusta/Richmond County, GA/SC',
 'Austin/Round Rock/San Marcos, TX',
 'Bakersfield/Delano, CA',
 'Baltimore/Towson, MD',
 'Bangor, ME',
 'Barnstable Town, MA',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Beaumont/Port Arthur, TX',
 'Bellingham, WA',
 'Bend, OR',
 'Billings, MT',
 'Binghamton, NY',
 'Birmingham/Hoover, AL',
 'Bismarck, ND',
 'Blacksburg/Christiansburg/Radford, VA',
 'Bloomington, IN',
 'Bloomington/Normal, IL',
 'Boise City/Nampa, ID',
 'Boston/Cambridge/Quincy, MA/NH',
 'Boulder, CO /1',
 'Bowling Green, KY

In [0]:
only_msa.sort_values('MSA')['MSA'].tolist()

['Abilene, TX',
 'Akron, OH',
 'Albany, GA',
 'Albany/Schenectady/Troy, NY',
 'Albuquerque, NM',
 'Alexandria, LA',
 'Allentown/Bethlehem/Easton, PA',
 'Altoona, PA',
 'Amarillo, TX',
 'Ann Arbor, MI',
 'Anniston, AL',
 'Appleton/Oshkosh/Neenah, WI',
 'Asheville, NC',
 'Athens, GA',
 'Atlanta, GA',
 'Atlantic/Cape May, NJ',
 'Auburn/Opelika, AL',
 'Augusta/Aiken, GA/SC',
 'Austin/San Marcos, TX',
 'Bakersfield, CA',
 'Baltimore, MD',
 'Bangor, ME',
 'Barnstable/Yarmouth, MA',
 'Baton Rouge, LA',
 'Beaumont/Port Arthur, TX',
 'Bellingham, WA',
 'Benton Harbor, MI',
 'Bergen/Passaic, NJ',
 'Billings, MT',
 'Biloxi/Gulfport/Pascagoula, MS',
 'Binghamton, NY',
 'Birmingham, AL',
 'Bismarck, ND',
 'Bloomington, IN',
 'Boise City, ID',
 'Boston, MA/NH',
 'Boulder/Longmont, CO',
 'Brazoria, TX',
 'Bremerton, WA',
 'Bridgeport, CT',
 'Brockton, MA',
 'Brownsville/Harlingen/San Benito, TX',
 'Bryan/College Station, TX',
 'Buffalo/Niagara Falls, NY',
 'Burlington, VT',
 'Canton/Massillon, OH',
 

In [0]:
remove_pop = ['Ames, IA','Anderson, IN','Anderson, SC', 'Appleton, WI', 'Battle Creek, MI',
 'Bay City, MI', 'Bend, OR', 'Blacksburg/Christiansburg/Radford, VA', 'Bowling Green, KY', 
             'Brunswick, GA', 'Burlington, NC', 'Cape Coral/Fort Myers, FL', 'Cape Girardeau/Jackson, MO/IL',
              'Carson City, NV', 'Cleveland, TN', "Coeur d'Alene, ID",'College Station/Bryan, TX',
             'Columbus, IN', 'Crestview/Fort Walton Beach/Destin, FL', 'Dalton, GA','Danville, IL',
             'Deltona/Daytona Beach/Ormond Beach, FL', 'Durham/Chapel Hill, NC','El Centro, CA', 
             'Elizabethtown, KY', 'El Paso, TX','Fairbanks, AK', 'Farmington, NM', 'Florence, SC',
 'Florence/Muscle Shoals, AL',  'Fond du Lac, WI', 'Gainesville, GA', 'Gulfport/Biloxi, MS','Hanford/Corcoran, CA',
             'Harrisonburg, VA', 'Hinesville/Fort Stewart, GA','Holland/Grand Haven, MI', 'Hot Springs, AR',
              'Idaho Falls, ID', 'Ithaca, NY', 'Jefferson City, MO','Kingsport/Bristol/Bristol, TN/VA',
 'Kingston, NY', 'Lake Havasu City/Kingman, AZ','Lebanon, PA','Lewiston, ID/WA','Logan, UT/ID', 'Longview, WA','Madera/Chowchilla, CA',
              'Manhattan, KS','Mankato/North Mankato, MN','Michigan City/La Porte, IN',
 'Midland, TX', 'Monroe, MI', 'Morgantown, WV','Morristown, TN','Mount Vernon/Anacortes, WA', 'Muskegon/Norton Shores, MI',
             'Napa, CA', 'Niles/Benton Harbor, MI','North Port/Bradenton/Sarasota, FL','Norwich/New London, CT',
              'Ocean City, NJ', 'Ogden/Clearfield, UT', 'Oshkosh/Neenah, WI', 'Oxnard/Thousand Oaks/Ventura, CA',
 'Palm Bay/Melbourne/Titusville, FL', 'Palm Coast, FL', 'Pascagoula, MS', 'Port St. Lucie, FL','Poughkeepsie/Newburgh/Middletown, NY', 'Prescott, AZ',
              'Valdosta, GA', 'Virginia Beach/Norfolk/Newport News, VA/NC', 'Warner Robins, GA', 'Wenatchee/East Wenatchee, WA',
           'Rome, GA', 'Sandusky, OH', 'Sebastian/Vero Beach, FL','Spartanburg, SC', 'Springfield, OH','Kennewick/Pasco/Richland, WA','St. George, UT', 'Winchester, VA/WV','Salisbury, MD',
             'Winston/Salem, NC']
remove_msa = ['Benton Harbor, MI','Bergen/Passaic, NJ','Brazoria, TX', 'Brockton, MA','Bryan/College Station, TX','Danbury, CT',
             'Daytona Beach, FL','Dutchess County, NY','El Paso, TX','Enid, OK', 'Florence, AL', 'Florence, SC',
             'Fort Lauderdale, FL','Fort Myers/Cape Coral, FL','Fort Pierce/Port St. Lucie, FL', 'Fort Walton Beach, FL', 'Fort Worth/Arlington, TX',
              'Galveston/Texas City, TX', 'Gary, IN','Hamilton/Middletown, OH','Jamestown, NY','Jersey City, NJ','Kenosha, WI',
              'Lawrence, MA/NH','Lowell, MA/NH', 'Melbourne/Titusville/Palm Bay, FL', 'Monmouth/Ocean, NJ','Nashua, NH','Nassau/Suffolk, NY', 'New London/Norwich, CT/RI',
 'New Bedford, MA', 'Newark, NJ','Newburgh, NY/PA','Norfolk/Virginia Beach/Newport News, VA/NC','Oakland, CA', 'Orange County, CA', 
              'Portsmouth/Rochester, NH/ME', 'Richland/Kennewick/Pasco, WA','Sharon, PA','Stamford/Norwalk, CT','Ventura, CA', 'Waterbury, CT', 'West Palm Beach/Boca Raton, FL', 'Wilmington/Newark, DE/MD',
              'Yolo, CA','Middlesex/Somerset/Hunterdon, NJ', 'Appleton/Oshkosh/Neenah, WI', 'Biloxi/Gulfport/Pascagoula, MS', 'Fitchburg-Leominster, MA', 'Sarasota/Bradenton, FL', 'Tacoma, WA'
              
]
#'Durham/Chapel Hill, NC' merge with raleigh

In [0]:
clean_pop_gdp = pop_gdp_merged.set_index('MSA').drop(remove_pop).reset_index().sort_values('MSA')
clean_only_msa = only_msa.set_index('MSA').drop(remove_msa)


In [0]:
clean_pop_gdp['MSA'] = clean_only_msa.reset_index().sort_values('MSA')['MSA']


In [0]:
clean_msa5 = clean_only_msa.merge(clean_pop_gdp, on = "MSA", how = "left")
clean_msa5

,MSA,index,MSA_Code,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010),Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007)
0,"Abilene, TX",0,40.0,16951.0,19767.0,20822.0,20170.0,3.124590471,0.159867
1,"Akron, OH",3,160.0,110875.0,147622.0,104590.0,99360.0,1.185679751,0.087970
2,"Albany, GA",7,240.0,12897.0,16963.0,16592.0,16928.0,-0.332630058,0.000753
3,"Albany/Schenectady/Troy, NY",2121,960.0,164967.0,224332.0,121318.0,132984.0,5.429514152,0.011663
4,"Albuquerque, NM",27,600.0,129503.0,189701.0,105264.0,110622.0,21.5758536,0.049326
5,"Alexandria, LA",11,220.0,13205.0,17433.0,18338.0,19538.0,6.127486469,0.118305
6,"Allentown/Bethlehem/Easton, PA",39,720.0,92279.0,139162.0,89242.0,94560.0,10.9101223,-0.013964
7,"Altoona, PA",15,280.0,12268.0,17955.0,19546.0,19294.0,-1.59124698,0.076935
8,"Amarillo, TX",33,640.0,28356.0,37755.0,33402.0,35108.0,10.31202267,0.104980
9,"Ann Arbor, MI",60,1320.0,134028.0,180477.0,84196.0,82856.0,19.15364278,0.146597


In [0]:
clean_msa6 = clean_msa5.set_index('MSA')
clean_msa6

,index,MSA_Code,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010),Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007)
MSA,,,,,,,,
"Abilene, TX",0,40.0,16951.0,19767.0,20822.0,20170.0,3.124590471,0.159867
"Akron, OH",3,160.0,110875.0,147622.0,104590.0,99360.0,1.185679751,0.087970
"Albany, GA",7,240.0,12897.0,16963.0,16592.0,16928.0,-0.332630058,0.000753
"Albany/Schenectady/Troy, NY",2121,960.0,164967.0,224332.0,121318.0,132984.0,5.429514152,0.011663
"Albuquerque, NM",27,600.0,129503.0,189701.0,105264.0,110622.0,21.5758536,0.049326
"Alexandria, LA",11,220.0,13205.0,17433.0,18338.0,19538.0,6.127486469,0.118305
"Allentown/Bethlehem/Easton, PA",39,720.0,92279.0,139162.0,89242.0,94560.0,10.9101223,-0.013964
"Altoona, PA",15,280.0,12268.0,17955.0,19546.0,19294.0,-1.59124698,0.076935
"Amarillo, TX",33,640.0,28356.0,37755.0,33402.0,35108.0,10.31202267,0.104980


In [0]:
clean_msa7 = clean_msa6.copy()
clean_msa7['Education change'] = (clean_msa6["Bachelor's degree or higher, 2013-17"] - clean_msa6["Bachelor's degree or higher, 2000"])/clean_msa6["Bachelor's degree or higher, 2000"]
clean_msa7['Enterprises change'] = (clean_msa6["Total Number of Enterprises (2010)"] - clean_msa6["Total Number of Enterprises (2000)"])/clean_msa6["Total Number of Enterprises (2000)"]
clean_msa7.drop(['index', 'MSA_Code'], inplace = True, axis =1)

In [0]:
clean_msa7

,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010),Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007),Education change,Enterprises change
MSA,,,,,,,,
"Abilene, TX",16951.0,19767.0,20822.0,20170.0,3.124590471,0.159867,0.166126,-0.031313
"Akron, OH",110875.0,147622.0,104590.0,99360.0,1.185679751,0.087970,0.331427,-0.050005
"Albany, GA",12897.0,16963.0,16592.0,16928.0,-0.332630058,0.000753,0.315267,0.020251
"Albany/Schenectady/Troy, NY",164967.0,224332.0,121318.0,132984.0,5.429514152,0.011663,0.359860,0.096161
"Albuquerque, NM",129503.0,189701.0,105264.0,110622.0,21.5758536,0.049326,0.464839,0.050901
"Alexandria, LA",13205.0,17433.0,18338.0,19538.0,6.127486469,0.118305,0.320182,0.065438
"Allentown/Bethlehem/Easton, PA",92279.0,139162.0,89242.0,94560.0,10.9101223,-0.013964,0.508057,0.059591
"Altoona, PA",12268.0,17955.0,19546.0,19294.0,-1.59124698,0.076935,0.463564,-0.012893
"Amarillo, TX",28356.0,37755.0,33402.0,35108.0,10.31202267,0.104980,0.331464,0.051075


In [0]:
clean_msa8_test = clean_msa7.dropna()
clean_msa8_test

,"Bachelor's degree or higher, 2000","Bachelor's degree or higher, 2013-17",Total Number of Enterprises (2000),Total Number of Enterprises (2010),Population Change From 2000 to 2010 (%),Percent Change in GDP (2001 to 2007),Education change,Enterprises change
MSA,,,,,,,,
"Abilene, TX",16951.0,19767.0,20822.0,20170.0,3.124590471,0.159867,0.166126,-0.031313
"Akron, OH",110875.0,147622.0,104590.0,99360.0,1.185679751,0.087970,0.331427,-0.050005
"Albany, GA",12897.0,16963.0,16592.0,16928.0,-0.332630058,0.000753,0.315267,0.020251
"Albany/Schenectady/Troy, NY",164967.0,224332.0,121318.0,132984.0,5.429514152,0.011663,0.359860,0.096161
"Albuquerque, NM",129503.0,189701.0,105264.0,110622.0,21.5758536,0.049326,0.464839,0.050901
"Alexandria, LA",13205.0,17433.0,18338.0,19538.0,6.127486469,0.118305,0.320182,0.065438
"Allentown/Bethlehem/Easton, PA",92279.0,139162.0,89242.0,94560.0,10.9101223,-0.013964,0.508057,0.059591
"Altoona, PA",12268.0,17955.0,19546.0,19294.0,-1.59124698,0.076935,0.463564,-0.012893
"Amarillo, TX",28356.0,37755.0,33402.0,35108.0,10.31202267,0.104980,0.331464,0.051075


In [0]:
cols_to_drop = [0, 1, 2, 3]
clean_msa9_test = clean_msa8_test.drop(clean_msa8_test.columns[cols_to_drop], axis=1)
clean_msa10_test = clean_msa9_test.rename(columns={"Population Change From 2000 to 2010 (%)": "Population Change", 
                                                  "Percent Change in GDP (2001 to 2007)":"Change in GDP", 
                                                  "Education Change":"Change of Individuals with Bachelors' Degrees", 
                                                  "Enterprises Change":"Change of Number of Enterprises"})


lst_pop = str_to_float(clean_msa10_test["Population Change"])

clean_msa10_test["Population Change"] = lst_pop

clean_msa11_test = clean_msa10_test

clean_msa11_test["Population Change"] = clean_msa11_test["Population Change"] / 100

clean_msa12_test = clean_msa11_test

clean_msa12_test






,Population Change,Change in GDP,Education change,Enterprises change
MSA,,,,
"Abilene, TX",0.031246,0.159867,0.166126,-0.031313
"Akron, OH",0.011857,0.087970,0.331427,-0.050005
"Albany, GA",-0.003326,0.000753,0.315267,0.020251
"Albany/Schenectady/Troy, NY",0.054295,0.011663,0.359860,0.096161
"Albuquerque, NM",0.215759,0.049326,0.464839,0.050901
"Alexandria, LA",0.061275,0.118305,0.320182,0.065438
"Allentown/Bethlehem/Easton, PA",0.109101,-0.013964,0.508057,0.059591
"Altoona, PA",-0.015912,0.076935,0.463564,-0.012893
"Amarillo, TX",0.103120,0.104980,0.331464,0.051075


In [0]:
training_data =clean_msa12_test.loc[established + declined]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [0]:
training_data['Established indicator'] = np.append(np.ones(13), np.zeros(14))
training_data2 = training_data.dropna()
training_data2

,Population Change,Change in GDP,Education change,Enterprises change,Established indicator
MSA,,,,,
"Raleigh/Durham/Chapel Hill, NC",0.124343,-0.021470,0.844925,0.198460,1.0
"San Diego, CA",0.042320,0.028480,0.568093,0.129163,1.0
"New York, NY",0.212035,0.054711,0.423402,0.052386,1.0
"Washington, DC/MD/VA/WV",0.033323,0.223857,0.521292,0.187083,1.0
"Dallas, TX",0.095019,0.733675,0.671146,0.108186,1.0
"Minneapolis/St. Paul, MN/WI",0.214857,0.239004,0.500701,0.065752,1.0
"Denver, CO",0.054584,0.056378,0.623811,0.076912,1.0
"Atlanta, GA",0.159334,0.022971,0.677168,0.157562,1.0
"Austin/San Marcos, TX",0.114458,-0.014522,1.014428,0.309188,1.0


In [0]:
test_data_final = clean_msa12_test.drop(index = training_data2.index)
test_data_final

,Population Change,Change in GDP,Education change,Enterprises change
MSA,,,,
"Abilene, TX",0.031246,0.159867,0.166126,-0.031313
"Albany, GA",-0.003326,0.000753,0.315267,0.020251
"Albany/Schenectady/Troy, NY",0.054295,0.011663,0.359860,0.096161
"Albuquerque, NM",0.215759,0.049326,0.464839,0.050901
"Alexandria, LA",0.061275,0.118305,0.320182,0.065438
"Allentown/Bethlehem/Easton, PA",0.109101,-0.013964,0.508057,0.059591
"Altoona, PA",-0.015912,0.076935,0.463564,-0.012893
"Amarillo, TX",0.103120,0.104980,0.331464,0.051075
"Ann Arbor, MI",0.191536,0.146597,0.346562,-0.015915


In [0]:
#Training model on the training set

X_train = training_data2.iloc[:, 0:4]
y_train = training_data2["Established indicator"]

lr_model = LogisticRegression().fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
#Listing out training accuracy

train_predictions = (lr_model.predict_proba(X_train)[:, 1] >= 0.56).astype(int)
train_predictions

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0])

In [0]:
accuracy_score(y_train, train_predictions)

0.7

In [0]:
#Testing model

X_test = test_data_final.iloc[:, 0:4]


test_predictions = (lr_model.predict_proba(X_test)[:, 1] >= 0.56).astype(int)
(test_predictions == 0).sum()

124

In [0]:
test_data_final["Predicted indicator"] = test_predictions
test_data_final2 = test_data_final
test_data_final3 = test_data_final2.drop(columns="Predicted indicator")
test_data_final3

,Population Change,Change in GDP,Education change,Enterprises change
MSA,,,,
"Abilene, TX",0.031246,0.159867,0.166126,-0.031313
"Albany, GA",-0.003326,0.000753,0.315267,0.020251
"Albany/Schenectady/Troy, NY",0.054295,0.011663,0.359860,0.096161
"Albuquerque, NM",0.215759,0.049326,0.464839,0.050901
"Alexandria, LA",0.061275,0.118305,0.320182,0.065438
"Allentown/Bethlehem/Easton, PA",0.109101,-0.013964,0.508057,0.059591
"Altoona, PA",-0.015912,0.076935,0.463564,-0.012893
"Amarillo, TX",0.103120,0.104980,0.331464,0.051075
"Ann Arbor, MI",0.191536,0.146597,0.346562,-0.015915


In [0]:
training_data3 = training_data2.rename(columns = {'Established indicator': "Predicted indicator"})
combined_df = test_data_final2.append(training_data3)
combined_df

,Population Change,Change in GDP,Education change,Enterprises change,Predicted indicator
MSA,,,,,
"Abilene, TX",0.031246,0.159867,0.166126,-0.031313,0.0
"Albany, GA",-0.003326,0.000753,0.315267,0.020251,0.0
"Albany/Schenectady/Troy, NY",0.054295,0.011663,0.359860,0.096161,0.0
"Albuquerque, NM",0.215759,0.049326,0.464839,0.050901,0.0
"Alexandria, LA",0.061275,0.118305,0.320182,0.065438,0.0
"Allentown/Bethlehem/Easton, PA",0.109101,-0.013964,0.508057,0.059591,0.0
"Altoona, PA",-0.015912,0.076935,0.463564,-0.012893,0.0
"Amarillo, TX",0.103120,0.104980,0.331464,0.051075,0.0
"Ann Arbor, MI",0.191536,0.146597,0.346562,-0.015915,0.0


In [0]:

df2 = pd.DataFrame()
df2["MSA"] = combined_df.index
df2["Predictions"] = combined_df["Predicted indicator"].tolist()

df3 = df2

df3

,MSA,Predictions
0,"Abilene, TX",0.0
1,"Albany, GA",0.0
2,"Albany/Schenectady/Troy, NY",0.0
3,"Albuquerque, NM",0.0
4,"Alexandria, LA",0.0
5,"Allentown/Bethlehem/Easton, PA",0.0
6,"Altoona, PA",0.0
7,"Amarillo, TX",0.0
8,"Ann Arbor, MI",0.0
9,"Anniston, AL",0.0


In [0]:
df4 = clean_msa2.loc[:, ["MSA", "County"]]
df4

,MSA,County
4,"Abilene, TX",Taylor
7,"Akron, OH",Portage
10,"Akron, OH",Summit
12,"Albany, GA",Dougherty
16,"Albany, GA",Lee
25,"Albany/Schenectady/Troy, NY",Schoharie
26,"Albany/Schenectady/Troy, NY",Schenectady
27,"Albany/Schenectady/Troy, NY",Saratoga
38,"Albany/Schenectady/Troy, NY",Montgomery
45,"Albany/Schenectady/Troy, NY",Rensselaer


In [0]:
predictions_w_county = df4.merge(df3, on = "MSA", how = "left")
predictions_w_county2 = predictions_w_county.dropna()
predictions_w_county2[predictions_w_county2["County"] == "Suffolk"]

,MSA,County,Predictions


In [0]:
from google.colab import files
predictions_w_county2.to_csv('finaldf.csv')
files.download('finaldf.csv')

In [0]:
print("Established:", sum(combined_df['Predicted indicator'] ==1))
print("Declined:", sum(combined_df['Predicted indicator'] ==0))

In [0]:
state_pred = pred_county.groupby('State_y').sum()
state_pred.sort_values("Predictions", ascending= False).reset_index()